In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import datetime,gc,math
import random
%matplotlib inline
import gc
import lightgbm as lgb
from matplotlib import pyplot as plt
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder,MinMaxScaler, StandardScaler
from sklearn.metrics import recall_score, auc, accuracy_score, f1_score, precision_score, classification_report, roc_auc_score,mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.feature_selection import VarianceThreshold

In [2]:
def active_time_transform(df):
    df['role_created_active_time'].fillna('"0-8,0","8-12,0","12-14,0","14-18,0","18-24,0"', inplace=True)
    df['role_created_active_time'] = df['role_created_active_time'].apply(lambda x:str(x)[1:-1])
    temp = df['role_created_active_time'].str.split(',',expand=True).iloc[:,[1,3,5,7,9]].rename(columns={1:'active_0-8',
                                        3:'active_8-12',5:'active_12-14',7:'active_14-18',9:'active_18-24'})
    for i in range(len(temp.columns)):
        temp.iloc[:,i] = temp.iloc[:,i].apply(lambda x:str(x)[:-1])
    df = df.join(temp).drop(['role_created_active_time'], axis=1)
    return df

def pay_grade_transform(df):
    df['pay_grade'].fillna('[0,0,0,0,0,0,0]',inplace=True)
    df['pay_grade'] = df['pay_grade'].apply(lambda x:str(x)[1:-1])
    temp = df['pay_grade'].str.split(',',expand=True).rename(columns={0:'pay_grade_1',
                1:'pay_grade_2',2:'pay_grade_3',3:'pay_grade_4',4:'pay_grade_5',5:'pay_grade_6',6:'pay_grade_7'})
    df = df.join(temp).drop(['pay_grade'], axis=1)
    return df

In [3]:
%%time
role_info = pd.read_csv('./data/mr_role_7d.csv')
role_info.drop_duplicates(subset=['user_id','cp_server_no','cp_role_id'],inplace=True)
role_info = active_time_transform(role_info)
role_info = pay_grade_transform(role_info)

Wall time: 2min 23s


In [4]:
# 删除2020-7-19日前30天的数据
role_info['create_role_time'] = pd.to_datetime(role_info['create_role_time'], format='%Y-%m-%d %H:%M:%S')
role_info.drop(role_info[role_info['create_role_time'] > (role_info['create_role_time'].max() + 
                                                          datetime.timedelta(-30))].index,inplace=True)

In [5]:
# 删除没有心跳数据的样本(1、没有登录;2、没有拿到心跳)
role_info.dropna(subset=['role_created_login_num'], inplace=True)
role_info.dropna(subset=['role_created_online'], inplace=True)
role_info['pay_num'].fillna(0, inplace=True)
role_info['pay_sum'].fillna(0, inplace=True)
role_info['role_created_active'] = role_info['role_created_active'].clip(0,7)
role_info['pay_rate'] = role_info['pay_num'] / (role_info['role_created_active'] + 1e-4)
role_info['pay_avg'] = role_info['pay_sum'] / (role_info['pay_num'] + 1e-4)

In [6]:
role_created_30_pay_sum = pd.read_csv('./data/role_created_30_pay_sum.csv',index_col=0)
# 对连接的关键字进行字符化或者整型化的调整
role_created_30_pay_sum['user_id'] = role_created_30_pay_sum['user_id'].astype(str)
role_created_30_pay_sum['mgame_id'] = role_created_30_pay_sum['mgame_id'].astype(str)
role_created_30_pay_sum['cp_server_no'] = role_created_30_pay_sum['cp_server_no'].astype(str)
role_created_30_pay_sum['cp_role_id'] = role_created_30_pay_sum['cp_role_id'].astype(str)
role_info['user_id'] = role_info['user_id'].astype(str)
role_info['mgame_id'] = role_info['mgame_id'].astype(str)
role_info['cp_server_no'] = role_info['cp_server_no'].astype(str)
role_info['cp_role_id'] = role_info['cp_role_id'].astype(str)
role_created_30_pay_sum.drop_duplicates(subset=['user_id','cp_server_no','mgame_id','cp_role_id'],inplace=True)
role_info.drop_duplicates(subset=['user_id','cp_server_no','mgame_id','cp_role_id'],inplace=True)
role_info = pd.merge(role_info,role_created_30_pay_sum,on=['user_id','mgame_id','cp_server_no','cp_role_id'],how='left',validate='one_to_one')

In [7]:
select_features = ['user_id','cp_server_no','cp_role_id','role_created_login_num',
       'role_created_active', 'role_created_online',
       'max_role_level', 'ip_num',
       'pay_num', 'pay_sum', 'active_0-8', 'active_8-12', 'active_12-14',
       'active_14-18', 'active_18-24', 'pay_grade_1', 'pay_grade_2',
       'pay_grade_3', 'pay_grade_4', 'pay_grade_5', 'pay_grade_6',
       'pay_rate', 'pay_avg','role_created_30_pay_sum']
role_info = role_info[select_features]
role_info['role_created_30_pay_sum'].fillna(0,inplace=True)
col_list = ['active_0-8','active_8-12','active_12-14','active_14-18','active_18-24','pay_grade_1','pay_grade_2',
            'pay_grade_3','pay_grade_4','pay_grade_5','pay_grade_6']
for col in col_list:
    role_info[col] = pd.to_numeric(role_info[col], errors='coerce')

In [8]:
role_info.to_pickle('./role_info_7d.pkl')

In [9]:
df_train, df_test = train_test_split(role_info,test_size=0.3)
df_train_pay = df_train[df_train['pay_sum']>0]
df_train_nopay = df_train[df_train['pay_sum']==0]

In [10]:
target = df_train_pay['role_created_30_pay_sum']
features = df_train_pay.drop(['role_created_30_pay_sum','user_id','cp_server_no','cp_role_id'], axis=1)
target_ln = np.log1p(target)
X_val, x_test, Y_val, y_test = train_test_split(features,target_ln, test_size=0.3)

In [11]:
# lgb的调参与交叉验证
train_data = lgb.Dataset(X_val, label=Y_val)
val_data = lgb.Dataset(x_test, label=y_test, reference=train_data)

# 设置参数初始值，不含交叉验证参数
print('设置参数')
params = {
    'boosting_type':'gbdt',
    'objective':'regression',
    'metric':'rmse',
    'nthread':4,
    'learning_rate':0.05
}

# 交叉验证（调参）
print('交叉验证')
min_rmse = float('5')
best_params = {}

# 准确率
print('调参1：提高准确率')
for num_leaves in range(5, 100, 5):
    for max_depth in range(3, 8, 1):
        params['num_leaves'] = num_leaves
        params['max_depth'] = max_depth
        
        cv_results = lgb.cv(
                            params,
                            train_data,
                            nfold=3,
                            stratified=False,
                            early_stopping_rounds=10,
                            verbose_eval=50     
                            )
        mean_rmse = pd.Series(cv_results['rmse-mean']).min()
        boost_rounds = pd.Series(cv_results['rmse-mean']).idxmin()
        
        if mean_rmse <= min_rmse:
            min_rmse = mean_rmse
            best_params['num_leaves'] = num_leaves
            best_params['max_depth'] = max_depth

if 'num_leaves' and 'max_depth' in best_params.keys():
    params['num_leaves'] = best_params['num_leaves']
    params['max_depth'] = best_params['max_depth']

# 过拟合
print('调参2：降低过拟合')
min_rmse = float('5')
for max_bin in range(5, 256, 10):
    for min_data_in_leaf in range(1, 102, 10):
        params['max_bin'] = max_bin
        params['min_data_in_leaf'] = min_data_in_leaf
        
        cv_results = lgb.cv(
                            params,
                            train_data,
                            nfold=3,
                            stratified=False,
                            early_stopping_rounds=10,
                            verbose_eval=50 
                            )
        mean_rmse = pd.Series(cv_results['rmse-mean']).min()
        boost_rounds = pd.Series(cv_results['rmse-mean']).idxmin()
        
        if mean_rmse <= min_rmse:
            min_rmse = mean_rmse
            best_params['min_data_in_leaf'] = min_data_in_leaf
            best_params['max_bin'] = max_bin
            
if 'max_bin' and 'min_data_in_leaf' in best_params.keys():
    params['max_bin'] = best_params['max_bin']
    params['min_data_in_leaf'] = best_params['min_data_in_leaf']

print('调参3：降低过拟合')
min_rmse = float('5')
for feature_fraction in [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
    for bagging_fraction in [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
        for bagging_freq in range(0,50,5):
            params['feature_fraction'] = feature_fraction
            params['bagging_fraction'] = bagging_fraction
            params['bagging_freq'] = bagging_freq

            cv_results = lgb.cv(
                            params,
                            train_data,
                            nfold=3,
                            stratified=False,
                            early_stopping_rounds=10,
                            verbose_eval=50
                            )
            mean_rmse = pd.Series(cv_results['rmse-mean']).min()
            boost_rounds = pd.Series(cv_results['rmse-mean']).idxmin()
        
            if mean_rmse <= min_rmse:
                min_rmse = mean_rmse
                best_params['feature_fraction'] = feature_fraction
                best_params['bagging_fraction'] = bagging_fraction
                best_params['bagging_freq'] = bagging_freq

if 'feature_fraction' and 'bagging_fraction' and 'bagging_freq' in best_params.keys():
    params['feature_fraction'] = best_params['feature_fraction']
    params['bagging_fraction'] = best_params['bagging_fraction']
    params['bagging_freq'] = best_params['bagging_freq']

print('调参4：降低过拟合')
min_rmse = float('5')
for lambda_l1 in [1e-5,1e-3,1e-1,0.0,0.1,0.3,0.5,0.7,0.9,1.0]:
    for lambda_l2 in [1e-5,1e-3,1e-1,0.0,0.1,0.4,0.6,0.7,0.9,1.0]:
        params['lambda_l1'] = lambda_l1
        params['lambda_l2'] = lambda_l2
        cv_results = lgb.cv(
                            params,
                            train_data,
                            nfold=3,
                            stratified=False,
                            early_stopping_rounds=10,
                            verbose_eval=50
                            )
        mean_rmse = pd.Series(cv_results['rmse-mean']).min()
        boost_rounds = pd.Series(cv_results['rmse-mean']).idxmin()
        
        if mean_rmse <= min_rmse:
            min_rmse = mean_rmse
            best_params['lambda_l1'] = lambda_l1
            best_params['lambda_l2'] = lambda_l2
if 'lambda_l1' and 'lambda_l2' in best_params.keys():
    params['lambda_l1'] = best_params['lambda_l1']
    params['lambda_l2'] = best_params['lambda_l2']

print("调参5：降低过拟合2")
min_rmse = float('5')
for min_split_gain in [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]:
    params['min_split_gain'] = min_split_gain
    
    cv_results = lgb.cv(
                            params,
                            train_data,
                            nfold=3,
                            stratified=False,
                            early_stopping_rounds=10,
                            verbose_eval=50    
                            )
    mean_rmse = pd.Series(cv_results['rmse-mean']).min()
    boost_rounds = pd.Series(cv_results['rmse-mean']).idxmin()
        
    if mean_rmse >= min_rmse:
        min_rmse = mean_rmse
        
    best_params['min_split_gain'] = min_split_gain
if 'min_split_gain' in best_params.keys():
    params['min_split_gain'] = best_params['min_split_gain']
print(best_params)

设置参数
交叉验证
调参1：提高准确率
[50]	cv_agg's rmse: 0.722277 + 0.00309399
[100]	cv_agg's rmse: 0.693101 + 0.00399823
[50]	cv_agg's rmse: 0.722277 + 0.00309399
[100]	cv_agg's rmse: 0.693103 + 0.00405848
[50]	cv_agg's rmse: 0.722277 + 0.00309399
[100]	cv_agg's rmse: 0.693103 + 0.00405848
[50]	cv_agg's rmse: 0.722277 + 0.00309399
[100]	cv_agg's rmse: 0.693103 + 0.00405848
[50]	cv_agg's rmse: 0.722277 + 0.00309399
[100]	cv_agg's rmse: 0.693103 + 0.00405848
[50]	cv_agg's rmse: 0.712164 + 0.00325714
[100]	cv_agg's rmse: 0.69194 + 0.00445435
[50]	cv_agg's rmse: 0.70959 + 0.00337296
[100]	cv_agg's rmse: 0.691386 + 0.00454767
[50]	cv_agg's rmse: 0.709595 + 0.00337468
[100]	cv_agg's rmse: 0.691204 + 0.00454376
[50]	cv_agg's rmse: 0.709595 + 0.00337468
[100]	cv_agg's rmse: 0.691323 + 0.00442205
[50]	cv_agg's rmse: 0.709595 + 0.00337468
[100]	cv_agg's rmse: 0.691291 + 0.00449924
[50]	cv_agg's rmse: 0.712164 + 0.00325714
[100]	cv_agg's rmse: 0.69194 + 0.00445435
[50]	cv_agg's rmse: 0.706668 + 0.00350526
[100]	

[50]	cv_agg's rmse: 0.7068 + 0.00349004
[100]	cv_agg's rmse: 0.691086 + 0.0044798
[50]	cv_agg's rmse: 0.706787 + 0.00349908
[100]	cv_agg's rmse: 0.690972 + 0.00447009
[50]	cv_agg's rmse: 0.706797 + 0.00350436
[100]	cv_agg's rmse: 0.691002 + 0.00449055
[50]	cv_agg's rmse: 0.706742 + 0.00349833
[100]	cv_agg's rmse: 0.690944 + 0.00449732
[50]	cv_agg's rmse: 0.706755 + 0.00348942
[100]	cv_agg's rmse: 0.690999 + 0.00436144
[50]	cv_agg's rmse: 0.706764 + 0.00349199
[100]	cv_agg's rmse: 0.690963 + 0.00443805
[50]	cv_agg's rmse: 0.706745 + 0.00352044
[100]	cv_agg's rmse: 0.690863 + 0.00441956
[50]	cv_agg's rmse: 0.70678 + 0.00351121
[100]	cv_agg's rmse: 0.690942 + 0.00450282
[50]	cv_agg's rmse: 0.70675 + 0.00349359
[100]	cv_agg's rmse: 0.690913 + 0.00448558
[50]	cv_agg's rmse: 0.706872 + 0.00350089
[100]	cv_agg's rmse: 0.691118 + 0.00449951
[50]	cv_agg's rmse: 0.706881 + 0.00352564
[100]	cv_agg's rmse: 0.691193 + 0.00446915
[50]	cv_agg's rmse: 0.7068 + 0.00349004
[100]	cv_agg's rmse: 0.691086 

[50]	cv_agg's rmse: 0.706872 + 0.00350089
[100]	cv_agg's rmse: 0.691118 + 0.00449951
[50]	cv_agg's rmse: 0.706881 + 0.00352564
[100]	cv_agg's rmse: 0.691193 + 0.00446915
[50]	cv_agg's rmse: 0.7068 + 0.00349004
[100]	cv_agg's rmse: 0.691086 + 0.0044798
[50]	cv_agg's rmse: 0.706787 + 0.00349908
[100]	cv_agg's rmse: 0.690972 + 0.00447009
[50]	cv_agg's rmse: 0.706797 + 0.00350436
[100]	cv_agg's rmse: 0.691002 + 0.00449055
[50]	cv_agg's rmse: 0.706742 + 0.00349833
[100]	cv_agg's rmse: 0.690944 + 0.00449732
[50]	cv_agg's rmse: 0.706755 + 0.00348942
[100]	cv_agg's rmse: 0.690999 + 0.00436144
[50]	cv_agg's rmse: 0.706764 + 0.00349199
[100]	cv_agg's rmse: 0.690963 + 0.00443805
[50]	cv_agg's rmse: 0.706745 + 0.00352044
[100]	cv_agg's rmse: 0.690863 + 0.00441956
[50]	cv_agg's rmse: 0.70678 + 0.00351121
[100]	cv_agg's rmse: 0.690942 + 0.00450282
[50]	cv_agg's rmse: 0.70675 + 0.00349359
[100]	cv_agg's rmse: 0.690913 + 0.00448558
[50]	cv_agg's rmse: 0.706872 + 0.00350089
[100]	cv_agg's rmse: 0.69111

[50]	cv_agg's rmse: 0.70678 + 0.00351121
[100]	cv_agg's rmse: 0.690942 + 0.00450282
[50]	cv_agg's rmse: 0.70675 + 0.00349359
[100]	cv_agg's rmse: 0.690913 + 0.00448558
[50]	cv_agg's rmse: 0.706872 + 0.00350089
[100]	cv_agg's rmse: 0.691118 + 0.00449951
[50]	cv_agg's rmse: 0.706881 + 0.00352564
[100]	cv_agg's rmse: 0.691193 + 0.00446915
[50]	cv_agg's rmse: 0.7068 + 0.00349004
[100]	cv_agg's rmse: 0.691086 + 0.0044798
[50]	cv_agg's rmse: 0.706787 + 0.00349908
[100]	cv_agg's rmse: 0.690972 + 0.00447009
[50]	cv_agg's rmse: 0.706797 + 0.00350436
[100]	cv_agg's rmse: 0.691002 + 0.00449055
[50]	cv_agg's rmse: 0.706742 + 0.00349833
[100]	cv_agg's rmse: 0.690944 + 0.00449732
[50]	cv_agg's rmse: 0.706755 + 0.00348942
[100]	cv_agg's rmse: 0.690999 + 0.00436144
[50]	cv_agg's rmse: 0.706764 + 0.00349199
[100]	cv_agg's rmse: 0.690963 + 0.00443805
[50]	cv_agg's rmse: 0.706745 + 0.00352044
[100]	cv_agg's rmse: 0.690863 + 0.00441956
[50]	cv_agg's rmse: 0.70678 + 0.00351121
[100]	cv_agg's rmse: 0.690942

[50]	cv_agg's rmse: 0.710171 + 0.00391213
[100]	cv_agg's rmse: 0.693365 + 0.00480517
[50]	cv_agg's rmse: 0.710191 + 0.00355305
[100]	cv_agg's rmse: 0.693226 + 0.0046661
[50]	cv_agg's rmse: 0.710256 + 0.00367556
[100]	cv_agg's rmse: 0.693464 + 0.00461669
[50]	cv_agg's rmse: 0.709564 + 0.00387355
[100]	cv_agg's rmse: 0.692044 + 0.00459669
[50]	cv_agg's rmse: 0.709487 + 0.00386349
[100]	cv_agg's rmse: 0.692387 + 0.00464958
[50]	cv_agg's rmse: 0.709733 + 0.00400964
[100]	cv_agg's rmse: 0.692722 + 0.00483458
[50]	cv_agg's rmse: 0.709784 + 0.00387709
[100]	cv_agg's rmse: 0.692733 + 0.00480159
[50]	cv_agg's rmse: 0.709579 + 0.00394335
[100]	cv_agg's rmse: 0.692932 + 0.00469213
[50]	cv_agg's rmse: 0.709781 + 0.0042481
[100]	cv_agg's rmse: 0.693121 + 0.00475354
[50]	cv_agg's rmse: 0.709943 + 0.00402744
[100]	cv_agg's rmse: 0.692893 + 0.00493068
[50]	cv_agg's rmse: 0.710001 + 0.00400275
[100]	cv_agg's rmse: 0.693 + 0.00457271
[50]	cv_agg's rmse: 0.710099 + 0.00393033
[100]	cv_agg's rmse: 0.69308

[50]	cv_agg's rmse: 0.708393 + 0.00371531
[100]	cv_agg's rmse: 0.691505 + 0.00470787
[50]	cv_agg's rmse: 0.708384 + 0.00407215
[100]	cv_agg's rmse: 0.691697 + 0.00467632
[50]	cv_agg's rmse: 0.708524 + 0.00384269
[100]	cv_agg's rmse: 0.691539 + 0.00447004
[50]	cv_agg's rmse: 0.708628 + 0.0038784
[100]	cv_agg's rmse: 0.69163 + 0.00490412
[50]	cv_agg's rmse: 0.708459 + 0.00390281
[100]	cv_agg's rmse: 0.691721 + 0.00468076
[50]	cv_agg's rmse: 0.708593 + 0.00393691
[100]	cv_agg's rmse: 0.691894 + 0.00460381
[50]	cv_agg's rmse: 0.708512 + 0.00383326
[100]	cv_agg's rmse: 0.691456 + 0.00462154
[50]	cv_agg's rmse: 0.708512 + 0.00383326
[100]	cv_agg's rmse: 0.691456 + 0.00462154
[50]	cv_agg's rmse: 0.708512 + 0.00383326
[100]	cv_agg's rmse: 0.691456 + 0.00462154
[50]	cv_agg's rmse: 0.708512 + 0.00383326
[100]	cv_agg's rmse: 0.691456 + 0.00462154
[50]	cv_agg's rmse: 0.708512 + 0.00383326
[100]	cv_agg's rmse: 0.691456 + 0.00462154
[50]	cv_agg's rmse: 0.708512 + 0.00383326
[100]	cv_agg's rmse: 0.69

[100]	cv_agg's rmse: 0.691225 + 0.00448715
[50]	cv_agg's rmse: 0.707197 + 0.00376598
[100]	cv_agg's rmse: 0.691256 + 0.00465287
[50]	cv_agg's rmse: 0.707297 + 0.00386198
[100]	cv_agg's rmse: 0.691577 + 0.00463492
[50]	cv_agg's rmse: 0.707482 + 0.00392681
[100]	cv_agg's rmse: 0.691781 + 0.00478294
[50]	cv_agg's rmse: 0.707249 + 0.0038078
[100]	cv_agg's rmse: 0.691854 + 0.0045374
[50]	cv_agg's rmse: 0.707539 + 0.00403169
[100]	cv_agg's rmse: 0.691752 + 0.0048994
[50]	cv_agg's rmse: 0.707859 + 0.00375457
[100]	cv_agg's rmse: 0.69189 + 0.00453652
[50]	cv_agg's rmse: 0.707898 + 0.00353928
[100]	cv_agg's rmse: 0.691838 + 0.00471138
[50]	cv_agg's rmse: 0.707458 + 0.0036289
[100]	cv_agg's rmse: 0.691735 + 0.00450196
[50]	cv_agg's rmse: 0.707622 + 0.00355766
[100]	cv_agg's rmse: 0.691969 + 0.00442324
[50]	cv_agg's rmse: 0.707375 + 0.00364321
[100]	cv_agg's rmse: 0.691225 + 0.00448715
[50]	cv_agg's rmse: 0.70727 + 0.00377877
[100]	cv_agg's rmse: 0.691206 + 0.00473527
[50]	cv_agg's rmse: 0.707537

[100]	cv_agg's rmse: 0.691051 + 0.0044708
[50]	cv_agg's rmse: 0.706995 + 0.00355721
[100]	cv_agg's rmse: 0.691051 + 0.0044708
[50]	cv_agg's rmse: 0.706995 + 0.00355721
[100]	cv_agg's rmse: 0.691051 + 0.0044708
[50]	cv_agg's rmse: 0.706745 + 0.00352044
[100]	cv_agg's rmse: 0.690863 + 0.00441956
[50]	cv_agg's rmse: 0.70632 + 0.00346829
[100]	cv_agg's rmse: 0.691743 + 0.00443111
[50]	cv_agg's rmse: 0.706327 + 0.00341066
[100]	cv_agg's rmse: 0.691768 + 0.00463341
[50]	cv_agg's rmse: 0.707081 + 0.00373815
[100]	cv_agg's rmse: 0.691973 + 0.00443371
[50]	cv_agg's rmse: 0.706529 + 0.00334724
[100]	cv_agg's rmse: 0.692035 + 0.00452301
[50]	cv_agg's rmse: 0.707344 + 0.00377705
[100]	cv_agg's rmse: 0.692359 + 0.00456467
[50]	cv_agg's rmse: 0.707488 + 0.00372299
[100]	cv_agg's rmse: 0.69224 + 0.00475435
[50]	cv_agg's rmse: 0.707763 + 0.00367925
[100]	cv_agg's rmse: 0.692182 + 0.00471891
[50]	cv_agg's rmse: 0.70736 + 0.00320426
[100]	cv_agg's rmse: 0.692319 + 0.00442388
[50]	cv_agg's rmse: 0.707719

[100]	cv_agg's rmse: 0.690875 + 0.00434188
[50]	cv_agg's rmse: 0.706679 + 0.00348439
[100]	cv_agg's rmse: 0.69081 + 0.00433638
[50]	cv_agg's rmse: 0.706775 + 0.00352523
[100]	cv_agg's rmse: 0.690832 + 0.00438581
[50]	cv_agg's rmse: 0.706726 + 0.00348804
[100]	cv_agg's rmse: 0.690803 + 0.00429585
[50]	cv_agg's rmse: 0.706736 + 0.00346389
[100]	cv_agg's rmse: 0.690813 + 0.00439476
[50]	cv_agg's rmse: 0.706759 + 0.00353743
[100]	cv_agg's rmse: 0.690973 + 0.00436085
[50]	cv_agg's rmse: 0.70661 + 0.00351666
[100]	cv_agg's rmse: 0.690833 + 0.00432671
[50]	cv_agg's rmse: 0.70661 + 0.00351665
[100]	cv_agg's rmse: 0.690833 + 0.00432671
[50]	cv_agg's rmse: 0.706714 + 0.00350365
[100]	cv_agg's rmse: 0.690777 + 0.00432724
[50]	cv_agg's rmse: 0.70661 + 0.00351666
[100]	cv_agg's rmse: 0.690833 + 0.00432671
[50]	cv_agg's rmse: 0.706714 + 0.00350365
[100]	cv_agg's rmse: 0.690777 + 0.00432724
[50]	cv_agg's rmse: 0.706702 + 0.00344457
[100]	cv_agg's rmse: 0.6908 + 0.00440302
[50]	cv_agg's rmse: 0.706652

In [12]:
params = {

'boosting_type':'gbdt',
'objective':'regression',
'metric':'rmse',
'nthread':4,
'learning_rate':0.05,
"max_depth":7,
"num_leaves":15,
"max_bin":255,
"min_data_in_leaf":81,
"min_split_gain":1.0,
"feature_fraction": 1,
"bagging_fraction":0.9,
"bagging_freq":30,
"lambda_l1":1.0,
"lambda_l2":0.0,
"metric": "rmse",
}
train_data = lgb.Dataset(X_val, label=Y_val)
val_data = lgb.Dataset(x_test, label=y_test, reference=train_data)
lgb_r_cv = lgb.train(params, train_data, num_boost_round=8000, early_stopping_rounds=100, valid_sets=[train_data, val_data])

[1]	training's rmse: 1.71939	valid_1's rmse: 1.71012
Training until validation scores don't improve for 100 rounds
[2]	training's rmse: 1.64879	valid_1's rmse: 1.63974
[3]	training's rmse: 1.58226	valid_1's rmse: 1.57339
[4]	training's rmse: 1.51967	valid_1's rmse: 1.51109
[5]	training's rmse: 1.46081	valid_1's rmse: 1.45245
[6]	training's rmse: 1.40557	valid_1's rmse: 1.3974
[7]	training's rmse: 1.35364	valid_1's rmse: 1.34561
[8]	training's rmse: 1.305	valid_1's rmse: 1.29704
[9]	training's rmse: 1.25934	valid_1's rmse: 1.2514
[10]	training's rmse: 1.21673	valid_1's rmse: 1.20885
[11]	training's rmse: 1.1768	valid_1's rmse: 1.16892
[12]	training's rmse: 1.13954	valid_1's rmse: 1.13165
[13]	training's rmse: 1.10483	valid_1's rmse: 1.09705
[14]	training's rmse: 1.07244	valid_1's rmse: 1.06472
[15]	training's rmse: 1.04234	valid_1's rmse: 1.03458
[16]	training's rmse: 1.01433	valid_1's rmse: 1.00658
[17]	training's rmse: 0.988305	valid_1's rmse: 0.980518
[18]	training's rmse: 0.964162	v

[152]	training's rmse: 0.682088	valid_1's rmse: 0.679226
[153]	training's rmse: 0.682088	valid_1's rmse: 0.679226
[154]	training's rmse: 0.682088	valid_1's rmse: 0.679226
[155]	training's rmse: 0.682088	valid_1's rmse: 0.679226
[156]	training's rmse: 0.682088	valid_1's rmse: 0.679226
[157]	training's rmse: 0.682088	valid_1's rmse: 0.679226
[158]	training's rmse: 0.682088	valid_1's rmse: 0.679226
[159]	training's rmse: 0.682088	valid_1's rmse: 0.679226
[160]	training's rmse: 0.682088	valid_1's rmse: 0.679226
[161]	training's rmse: 0.682088	valid_1's rmse: 0.679226
[162]	training's rmse: 0.682088	valid_1's rmse: 0.679226
[163]	training's rmse: 0.682088	valid_1's rmse: 0.679226
[164]	training's rmse: 0.682088	valid_1's rmse: 0.679226
[165]	training's rmse: 0.682088	valid_1's rmse: 0.679226
[166]	training's rmse: 0.682088	valid_1's rmse: 0.679226
[167]	training's rmse: 0.682088	valid_1's rmse: 0.679226
[168]	training's rmse: 0.682088	valid_1's rmse: 0.679226
[169]	training's rmse: 0.682088

In [13]:
df_test_pay = df_test[df_test['pay_sum']>0]
df_test_nopay = df_test[df_test['pay_sum']==0]

In [14]:
df_test_part1 = df_test_nopay[['user_id','cp_server_no','cp_role_id','pay_num']].rename(columns=
                                                                                {'pay_num':'predict_30_pay'})

In [15]:
target_test = df_test_pay['role_created_30_pay_sum']
target_test_ln = np.log1p(target_test)
features_test = df_test_pay.drop(['role_created_30_pay_sum','user_id','cp_server_no','cp_role_id'], axis=1)
y_predict = lgb_r_cv.predict(features_test)
y_predict[y_predict<0] = 0
mse = mean_squared_error(np.expm1(y_predict), np.expm1(target_test_ln))
mae = mean_absolute_error(np.expm1(y_predict), np.expm1(target_test_ln))
rmse = mse ** 0.5
print(rmse)
print(mse)
print(mae)

2271.9775383244805
5161881.934650966
409.89914744991177


In [16]:
sum(np.expm1(target_test_ln))/sum(np.expm1(y_predict))

1.2579590093700324

In [17]:
df_test_part2 = df_test_pay[['user_id','cp_server_no','cp_role_id']]
df_test_part2['predict_30_pay'] = np.expm1(y_predict) *1.255
pred = df_test_part1.append(df_test_part2)
predict_data = pd.merge(df_test[['user_id','cp_server_no','cp_role_id', 'role_created_30_pay_sum']],pred,on=['user_id','cp_server_no','cp_role_id'],how='left',validate='one_to_one')

In [18]:
mse = mean_squared_error(predict_data['predict_30_pay'], predict_data['role_created_30_pay_sum'])
mae = mean_absolute_error(predict_data['predict_30_pay'], predict_data['role_created_30_pay_sum'])
rmse = mse ** 0.5
print('测试集上的均方根误差:%.2f元'% rmse)
print('测试集上的平均绝对误差:%.2f元'% mae)
print('测试集前30天实际总的付费金额:%.2f元' % predict_data['role_created_30_pay_sum'].sum())
print('测试集前30天预测总的付费金额:%.2f元'% predict_data['predict_30_pay'].sum())
print('预测总金额准确率:',predict_data['predict_30_pay'].sum()/predict_data['role_created_30_pay_sum'].sum())

测试集上的均方根误差:337.28元
测试集上的平均绝对误差:10.90元
测试集前30天实际总的付费金额:30356348.00元
测试集前30天预测总的付费金额:29590017.36元
预测总金额准确率: 0.9747555060208736


In [19]:
import joblib
joblib.dump(lgb_r_cv,'./lgb_r_7d.pkl')

['./lgb_r_7d.pkl']